In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.spatial.distance import cosine as cs
from numpy.linalg import norm
import json


In [179]:
def get_input_data(input_path) :
    with open(input_path) as fin:
        inp = fin.read()
        inp_list = inp.split('\n')
        inp_list = list(filter(None, inp_list))

    return inp,inp_list 

In [99]:
with open('/Users/s0c02nj/Downloads/covid-19_research_collaboration-master/notebooks/output_fra.jsonl') as fin:
    output_jsonl = fin.read()

In [189]:
def get_sent_embed(input_path, output_jsonl) :
    
    document,list_doc = get_input_data(input_path)
    
    #We will run the model and get the outputs
    model_output = output_jsonl
    json_lines = model_output.split('\n')
    
    #getting the dimensions
    embed_size = 768
    
    #Defining the lists
    sent_embed = []
    tokens = []
    
    #Getting the final df
    df = pd.DataFrame()
    
    #Running for the sentence
    for i in range(0,len(list_doc)):
        line = json.loads(json_lines[i])
        
        #array for saving the embeddings
        feat_embed = np.zeros((len(line['features']),embed_size))
        
        #Temp list for saving the tokens
        token_temp = []
        for j,feature in enumerate(line['features']):
            
            token_temp.append(feature['token'])
            feat_embed[j] = feature['layers'][0]['values']
        
        #sanity checks
        avg_embed =  np.mean(feat_embed[1:len(feat_embed)-1],axis=0)
        if avg_embed.sum() == 0 :
            print ('Check_model')
        
        #final_output_embeddings
        sent_embed.append(avg_embed)
        tokens.append(' '.join(token_temp[1:len(token_temp)-1]))
        
         
        
    df['documents'] = tokens
    df['embedding'] = sent_embed
    
    return df
        

In [197]:
input_ans = '/Users/s0c02nj/Downloads/covid-19_research_collaboration-master/notebooks/answers.txt'

input_query = '/Users/s0c02nj/Downloads/covid-19_research_collaboration-master/notebooks/question.txt'

In [229]:
df_query['documents'].values[0]

'this is the first sentence'

In [230]:
def get_cosine_sim(input_ans, input_query ,output_jsonl, topk) :
    
    #We will run the model and get the outputs
    model_output = output_jsonl
    
    #Getting the dataframes
    df_query = get_sent_embed(input_query, model_output)
    df_ans =   get_sent_embed(input_ans, model_output)
    
    
    #Query embedding
    query_embed = df_query['embedding'].values[0]
    query_embed_norm = query_embed/norm(query_embed)
    
    #Answers embedding
    list_embed = df_ans['embedding'].tolist()
    
    #getting the answer embedding
    ans_embed =  np.stack(list_embed, axis=0)
    ans_embed_norm = ans_embed/norm(ans_embed,axis=1,keepdims= True)
    
    cos_sim = np.dot(ans_embed_norm,query_embed_norm)
    
    
    #Output Format
    df_out = pd.DataFrame()
    df_out['answers'] = df_ans['documents'].tolist()
    df_out['similarity'] = cos_sim
    df_out['Query'] = df_query['documents'].values[0]
    
    #Output sorted
    final_df = df_out.sort_values(by=['similarity'], ascending=False)[['Query','answers','similarity']]

    
    return final_df.head(topk)

In [233]:
final_df = get_cosine_sim(input_ans, input_query ,output_jsonl, topk=5)

In [234]:
final_df

,Query,answers,similarity
0,this is the first sentence,this is the first sentence,1.000000
1,this is the first sentence,this is the second sentence,0.951076
2,this is the first sentence,this is the third sentence,0.948371
3,this is the first sentence,this is not the third sentence,0.882423
4,this is the first sentence,the sky is blue this morning,0.399714
